In [19]:
!pip uninstall -y sentencepiece
!pip install sentencepiece==0.1.99


Found existing installation: sentencepiece 0.2.1
Uninstalling sentencepiece-0.2.1:
  Successfully uninstalled sentencepiece-0.2.1
   ---------------------------------------- 0.0/977.5 kB ? eta -:--:--
   --------------------- ------------------ 524.3/977.5 kB 5.6 MB/s eta 0:00:01
   ---------------------------------------- 977.5/977.5 kB 5.7 MB/s  0:00:00



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [22]:
import sys
print(">>> Python executable:", sys.executable)
print(">>> Python version:", sys.version)
try:
    import sentencepiece as spm
    print(">>> SentencePiece loaded OK")
except:
    print(">>> SentencePiece NOT FOUND")


>>> Python executable: c:\Users\thong\AppData\Local\Programs\Python\Python310\python.exe
>>> Python version: 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)]
>>> SentencePiece loaded OK


In [21]:
import torch
import os
from transformers import (
    T5ForConditionalGeneration, T5Tokenizer,
    MT5ForConditionalGeneration, MT5Tokenizer,
    AutoModelForSeq2SeqLM, AutoTokenizer
)
from peft import PeftModel
import torch.nn as nn
import math

import os

# ========================================
# ĐƯỜNG DẪN ĐÚNG THEO HÌNH CỦA BẠN
# ========================================
MODEL_DIR = r"C:\Users\thong\workplace\IUH\4RD_YEAR_COLLEGE\semester_1\deep learnning\CK\kaagle_Truong\results\models"

BARTPHO_PATH = os.path.join(MODEL_DIR, "bartpho", "final")
MT5_PATH = os.path.join(MODEL_DIR, "mt5-base", "final")
VIT5_PATH = os.path.join(MODEL_DIR, "vit5-base", "final")   # <-- FIXED


# ========================================
# 1. LOAD BARTPHO (Đơn giản nhất - ĐÃ ĐỦ FILE)
# ========================================
def load_bartpho(model_path):
    """Load BARTpho model từ local"""
    print(f"Loading BARTpho from: {model_path}")
    
    # Kiểm tra thư mục tồn tại
    if not os.path.exists(model_path):
        raise FileNotFoundError(f"Không tìm thấy thư mục: {model_path}")
    
    tokenizer = AutoTokenizer.from_pretrained(model_path, local_files_only=True)
    model = AutoModelForSeq2SeqLM.from_pretrained(
        model_path,
        local_files_only=True,
        torch_dtype=torch.float32  # Windows thường dùng float32
    )
    
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = model.to(device)
    model.eval()
    
    print(f"✅ BARTpho loaded on {device}")
    return model, tokenizer


# ========================================
# 2. LOAD mT5-Base (ĐÃ ĐỦ FILE)
# ========================================
def load_mt5(model_path):
    """Load mT5 model từ local"""
    print(f"Loading mT5 from: {model_path}")
    
    if not os.path.exists(model_path):
        raise FileNotFoundError(f"Không tìm thấy thư mục: {model_path}")
    
    tokenizer = MT5Tokenizer.from_pretrained(model_path, local_files_only=True)
    model = MT5ForConditionalGeneration.from_pretrained(
        model_path,
        local_files_only=True,
        torch_dtype=torch.float32
    )
    
    # Resize nếu vocab không khớp
    if len(tokenizer) != model.config.vocab_size:
        model.resize_token_embeddings(len(tokenizer))
    
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = model.to(device)
    model.eval()
    
    print(f"✅ mT5 loaded on {device}")
    return model, tokenizer


# ========================================
# 3. LOAD ViT5 (CẦN LOAD BASE + LoRA + Custom layers)
# ========================================

# Custom architecture classes (giữ nguyên như trước)
class CustomMultiHeadAttention(nn.Module):
    def __init__(self, hidden_size, num_heads, dropout=0.1):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_heads = num_heads
        self.head_dim = hidden_size // num_heads
        self.query = nn.Linear(hidden_size, hidden_size)
        self.key = nn.Linear(hidden_size, hidden_size)
        self.value = nn.Linear(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, hidden_size)
        self.dropout = nn.Dropout(dropout)
        self.scale = math.sqrt(self.head_dim)
        
    def forward(self, hidden_states, attention_mask=None):
        batch_size, seq_length, _ = hidden_states.size()
        Q = self.query(hidden_states).view(batch_size, seq_length, self.num_heads, self.head_dim).transpose(1, 2)
        K = self.key(hidden_states).view(batch_size, seq_length, self.num_heads, self.head_dim).transpose(1, 2)
        V = self.value(hidden_states).view(batch_size, seq_length, self.num_heads, self.head_dim).transpose(1, 2)
        scores = torch.matmul(Q, K.transpose(-2, -1)) / self.scale
        if attention_mask is not None:
            scores = scores.masked_fill(attention_mask == 0, -1e4)
        attn_weights = torch.softmax(scores, dim=-1)
        attn_weights = self.dropout(attn_weights)
        context = torch.matmul(attn_weights, V).transpose(1, 2).contiguous().view(batch_size, seq_length, self.hidden_size)
        return self.out(context), attn_weights

class BiLSTMLayer(nn.Module):
    def __init__(self, hidden_size, num_layers=2, dropout=0.1):
        super().__init__()
        self.lstm = nn.LSTM(hidden_size, hidden_size // 2, num_layers, 
                           dropout=dropout if num_layers > 1 else 0, bidirectional=True, batch_first=True)
        self.layer_norm = nn.LayerNorm(hidden_size)
        self.dropout = nn.Dropout(dropout)
    def forward(self, hidden_states):
        lstm_out, _ = self.lstm(hidden_states)
        return self.layer_norm(hidden_states + self.dropout(lstm_out))

class RNNLayer(nn.Module):
    def __init__(self, hidden_size, num_layers=1, dropout=0.1):
        super().__init__()
        self.rnn = nn.RNN(hidden_size, hidden_size, num_layers, 
                         dropout=dropout if num_layers > 1 else 0, batch_first=True)
        self.layer_norm = nn.LayerNorm(hidden_size)
        self.dropout = nn.Dropout(dropout)
    def forward(self, hidden_states):
        rnn_out, _ = self.rnn(hidden_states)
        return self.layer_norm(hidden_states + self.dropout(rnn_out))

class FeedForwardNetwork(nn.Module):
    def __init__(self, hidden_size, ff_size=None, dropout=0.1):
        super().__init__()
        ff_size = ff_size or hidden_size * 4
        self.fc1 = nn.Linear(hidden_size, ff_size)
        self.fc2 = nn.Linear(ff_size, hidden_size)
        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(hidden_size)
    def forward(self, hidden_states):
        residual = hidden_states
        x = torch.nn.functional.gelu(self.fc1(hidden_states))
        x = self.dropout(self.fc2(self.dropout(x)))
        return self.layer_norm(residual + x)

class T5WithAdvancedArchitecture(nn.Module):
    def __init__(self, base_model, hidden_size=768, num_heads=12):
        super().__init__()
        self.base_model = base_model
        self.config = base_model.config
        self.multi_head_attention = CustomMultiHeadAttention(hidden_size, num_heads)
        self.bilstm = BiLSTMLayer(hidden_size, num_layers=2)
        self.rnn = RNNLayer(hidden_size, num_layers=1)
        self.ffn = FeedForwardNetwork(hidden_size)
        self.layer_norm1 = nn.LayerNorm(hidden_size)
        self.layer_norm3 = nn.LayerNorm(hidden_size)
        self.dropout = nn.Dropout(0.1)
        self.gate = nn.Linear(hidden_size * 3, 3)
        
    def generate(self, *args, **kwargs):
        return self.base_model.generate(*args, **kwargs)


def load_vit5_advanced(adapter_path):
    """
    Load ViT5 với LoRA adapters và custom layers
    Vì bạn chỉ lưu adapters, cần download base model từ HuggingFace
    """
    print(f"Loading ViT5 from: {adapter_path}")
    
    if not os.path.exists(adapter_path):
        raise FileNotFoundError(f"Không tìm thấy thư mục: {adapter_path}")
    
    device = "cuda" if torch.cuda.is_available() else "cpu"
    
    # Step 1: Load tokenizer từ local (nếu có) hoặc từ HuggingFace
    tokenizer_path = adapter_path
    if os.path.exists(os.path.join(adapter_path, "tokenizer_config.json")):
        tokenizer = T5Tokenizer.from_pretrained(adapter_path, local_files_only=True)
    else:
        print("   ⚠️ Tokenizer không có local, downloading từ VietAI/vit5-base...")
        tokenizer = T5Tokenizer.from_pretrained("VietAI/vit5-base")
        # Add special tokens
        special_tokens = {"additional_special_tokens": ["<TIKTOK>", "<FACEBOOK>", "<YOUTUBE>", "<COMMENT>"]}
        tokenizer.add_special_tokens(special_tokens)
    
    # Step 2: Load base model từ HuggingFace
    print("   Loading base model VietAI/vit5-base...")
    base_model = T5ForConditionalGeneration.from_pretrained(
        "VietAI/vit5-base",
        torch_dtype=torch.float32
    )
    base_model.resize_token_embeddings(len(tokenizer))
    
    # Step 3: Load LoRA adapters
    print("   Loading LoRA adapters...")
    base_model = PeftModel.from_pretrained(base_model, adapter_path, local_files_only=True)
    base_model = base_model.merge_and_unload()  # Merge LoRA vào model
    
    # Step 4: Wrap với custom architecture
    model = T5WithAdvancedArchitecture(base_model, hidden_size=768, num_heads=12)
    
    # Step 5: Load custom layers
    custom_layers_path = os.path.join(adapter_path, "advanced_layers.pt")
    if os.path.exists(custom_layers_path):
        print("   Loading custom layers (BiLSTM, RNN, Attention)...")
        state_dict = torch.load(custom_layers_path, map_location=device)
        model.multi_head_attention.load_state_dict(state_dict['multi_head_attention'])
        model.bilstm.load_state_dict(state_dict['bilstm'])
        model.rnn.load_state_dict(state_dict['rnn'])
        model.ffn.load_state_dict(state_dict['ffn'])
        model.gate.load_state_dict(state_dict['gate'])
    else:
        print("   ⚠️ Không tìm thấy advanced_layers.pt!")
    
    model = model.to(device)
    model.eval()
    
    print(f"✅ ViT5 Advanced loaded on {device}")
    return model, tokenizer


# ========================================
# PREDICT FUNCTION
# ========================================
def predict(model, tokenizer, text, max_length=64, num_beams=4):
    """Predict với model"""
    device = next(model.parameters()).device
    
    inputs = tokenizer(
        text, 
        return_tensors="pt", 
        max_length=128, 
        truncation=True,
        padding=True
    ).to(device)
    
    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            max_length=max_length,
            num_beams=num_beams,
            early_stopping=True,
            no_repeat_ngram_size=2,
        )
    
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return result


# ========================================
# TEST
# ========================================
if __name__ == "__main__":
    
    # Test texts
    test_texts = [
        "<TIKTOK> video này hay quá đi mất",
        "<FACEBOOK> sản phẩm tốt lắm shop ơi", 
    ]
    
    # --- Test BARTpho ---
    print("\n" + "="*50)
    print("📌 Testing BARTpho")
    print("="*50)
    try:
        model, tokenizer = load_bartpho(BARTPHO_PATH)
        for text in test_texts:
            result = predict(model, tokenizer, text)
            print(f"Input:  {text}")
            print(f"Output: {result}\n")
    except Exception as e:
        print(f"❌ Error: {e}")
    
    # --- Test mT5 ---
    print("\n" + "="*50)
    print("📌 Testing mT5")
    print("="*50)
    try:
        model, tokenizer = load_mt5(MT5_PATH)
        for text in test_texts:
            result = predict(model, tokenizer, text)
            print(f"Input:  {text}")
            print(f"Output: {result}\n")
    except Exception as e:
        print(f"❌ Error: {e}")
    
    # --- Test ViT5 Advanced ---
    print("\n" + "="*50)
    print("📌 Testing ViT5 Advanced")
    print("="*50)
    try:
        model, tokenizer = load_vit5_advanced(VIT5_PATH)
        for text in test_texts:
            result = predict(model, tokenizer, text)
            print(f"Input:  {text}")
            print(f"Output: {result}\n")
    except Exception as e:
        print(f"❌ Error: {e}")


📌 Testing BARTpho
Loading BARTpho from: C:\Users\thong\workplace\IUH\4RD_YEAR_COLLEGE\semester_1\deep learnning\CK\kaagle_Truong\results\models\bartpho\final
❌ Error: 
BartphoTokenizer requires the SentencePiece library but it was not found in your environment. Checkout the instructions on the
installation page of its repo: https://github.com/google/sentencepiece#installation and follow the ones
that match your environment. Please note that you may need to restart your runtime after installation.


📌 Testing mT5
Loading mT5 from: C:\Users\thong\workplace\IUH\4RD_YEAR_COLLEGE\semester_1\deep learnning\CK\kaagle_Truong\results\models\mt5-base\final
❌ Error: 
MT5Tokenizer requires the SentencePiece library but it was not found in your environment. Checkout the instructions on the
installation page of its repo: https://github.com/google/sentencepiece#installation and follow the ones
that match your environment. Please note that you may need to restart your runtime after installation.


📌 

In [ ]:
# 1. Load model bạn muốn dùng
model, tokenizer = load_bartpho("/results/models/bartpho/final")
# HOẶC
model, tokenizer = load_mt5("/results/models/mt5-base/final")
# HOẶC  
model, tokenizer = load_vit5_advanced("/results/models/vit5-base/final")
# 2. Predict
result = predict(model, tokenizer, "<TIKTOK> video hay quá")
print(result)

# 3. Predict nhiều texts
results = predict_batch(model, tokenizer, ["text1", "text2", "text3"])